In [7]:
import pandas as pd
import glob
import pipeline_helper
from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble
import os
from scipy.signal import resample


In [2]:
def upsample_to_answer_shape(test_pred,
                             annot_len,
                             sampling_rate=1,
                             upsampling_factor=20,
                             lead_in_period_seconds=10,
                            ):
    test_pred = test_pred.reshape(1,-1)[0]
    duration = test_pred.shape[0]/sampling_rate
    upsampling_factor = 20
    new_sampling_rate = sampling_rate * upsampling_factor
    new_num_samples = duration * new_sampling_rate
    upsampled_signal = resample(test_pred, int(new_num_samples))
    start_test_period = lead_in_period_seconds*upsampling_factor
    test_period_test_pred_upsampled = upsampled_signal[start_test_period:(start_test_period+annot_len)]
    return test_period_test_pred_upsampled

# Scenario 2

In [110]:
scenario = 2
y_columns=['arousal']
for fold in range(5):
    print(fold)
    train_y_files = glob.glob(
        f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/train/annotations/*')
    train_x_files = glob.glob(
        f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/train/physiology/*1hz_zscored.csv')
    model = svm.SVR()
    x_columns = [ 
        'PPG_Rate', 
        'ECG_Rate','ECG_Quality','PPG_Peak_Height',
        'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
        'RSP_Amplitude', 'RSP_Rate',
        'corrugator_EMG_Clean',
        'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
        'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets', 'trapezius_EMG_Raw',
        'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
        'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
        'trapezius_EMG_Offsets', 'zygomaticus_EMG_Raw', 'zygomaticus_EMG_Clean',
        'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
        'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
        'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
        'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
        'sub_num',
        'vid_num',
    ]

    model = pipeline_helper.train_model_no_eval( 
        model,
        train_x_files,
        train_y_files,
        x_columns,
        y_columns,
        f'test_train_model_svr_scenario_{scenario}_fold_{fold}_fizz_{y_columns[0]}'
    )
    test_x_files = glob.glob(
    f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/test/physiology/*1hz.csv')

    os.makedirs(f'/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_{scenario}/fold_{fold}/test/annotations/',exist_ok=True)
    for test_x_file in test_x_files:
        file_name_split = test_x_file.split('/')[-1].split('_')
        sub, video = file_name_split[1],file_name_split[3]
        annote_path = f'annotations/sub_{sub}_vid_{video}.csv'
        annotation_file = f'/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_{scenario}/fold_{fold}/test/{annote_path}'
        if os.path.exists(annotation_file):
            annot_df = pd.read_csv(annotation_file)

        else:
            test_annotation_clean_path =test_x_file.split('phy')[0]+annote_path
            annot_df = pd.read_csv(test_annotation_clean_path)
        test_pred = model.predict(pd.read_csv(test_x_file)[x_columns].fillna(0).values)
        upsampled_preds = upsample_to_answer_shape(test_pred,len(annot_df))
        annot_df[y_columns[0]]=upsampled_preds
        annot_df.to_csv(annotation_file,index=None)

0


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


3


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


4


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [105]:
rsync -r /work/abslab/emognition_2023_challenge/data/scenario_2/fold_4/test/annotations/ /work/abslab/emognition_2023_challenge/results/test_submissions/scenario_2/fold_4




,Unnamed: 0.1,Unnamed: 0,time,valence,arousal
0,0,0,10000,5.073239,4.878170
1,1,1,10050,5.071915,4.877917
2,2,2,10100,5.070435,4.879012
3,3,3,10150,5.068822,4.881414
4,4,4,10200,5.067103,4.885060
...,...,...,...,...,...
596,596,596,39800,4.879948,5.135431
597,597,597,39850,4.876341,5.137386
598,598,598,39900,4.873217,5.138943
599,599,599,39950,4.870546,5.140130


In [22]:
scenario = 2
y_columns_list=[['arousal'],['valence']]
for y_columns in y_columns_list:
    for fold in range(5):
        print(fold)
        train_y_files = glob.glob(
            f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/train/annotations/*')
        train_x_files = glob.glob(
            f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/train/physiology/*1hz_zscored.csv')
        model = svm.SVR()
        x_columns = [ 
            'PPG_Rate', 
            'ECG_Rate','ECG_Quality','PPG_Peak_Height',
            'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
            'RSP_Amplitude', 'RSP_Rate',
            'corrugator_EMG_Clean',
            'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
            'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets', 'trapezius_EMG_Raw',
            'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
            'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
            'trapezius_EMG_Offsets', 'zygomaticus_EMG_Raw', 'zygomaticus_EMG_Clean',
            'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
            'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
            'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
            'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
            'sub_num',
            'vid_num',
        ]

        model = pipeline_helper.train_model_no_eval( 
            model,
            train_x_files,
            train_y_files,
            x_columns,
            y_columns,
            f'test_train_model_svr_scenario_{scenario}_fold_{fold}_fizz_{y_columns[0]}'
        )
        test_x_files = glob.glob(
        f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/test/physiology/*1hz.csv')
        os.makedirs(f'/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_{scenario}/fold_{fold}/test/annotations/',exist_ok=True)
        for test_x_file in test_x_files:
            file_name_split = test_x_file.split('/')[-1].split('_')
            sub, video = file_name_split[1],file_name_split[3]
            annote_path = f'annotations/sub_{sub}_vid_{video}.csv'
            annotation_file = f'/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_{scenario}/fold_{fold}/test/{annote_path}'
            if os.path.exists(annotation_file):
                annot_df = pd.read_csv(annotation_file)

            else:
                test_annotation_clean_path =test_x_file.split('phy')[0]+annote_path
                annot_df = pd.read_csv(test_annotation_clean_path)
            test_pred = model.predict(pd.read_csv(test_x_file)[x_columns].fillna(0).values)
            upsampled_preds = upsample_to_answer_shape(test_pred,len(annot_df))
            annot_df[y_columns[0]]=upsampled_preds
            annot_df.to_csv(annotation_file,index=None)


0


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2


...

/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


3


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


4


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
scenario = 3
y_columns=['valence']
for fold in range(4):
    print(fold)
    train_y_files = glob.glob(
        f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/train/annotations/*')
    train_x_files = glob.glob(
        f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/train/physiology/*1hz_zscored.csv')
    model = svm.SVR()
    x_columns = [ 
        'PPG_Rate', 
        'ECG_Rate','ECG_Quality','PPG_Peak_Height',
        'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
        'RSP_Amplitude', 'RSP_Rate',
        'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
        'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
        'sub_num',
        #'vid_num',
    ]

    model = pipeline_helper.train_model_no_eval( 
        model,
        train_x_files,
        train_y_files,
        x_columns,
        y_columns,
        f'test_train_model_svr_scenario_{scenario}_fold_{fold}_fizz_{y_columns[0]}'
    )
    test_x_files = glob.glob(
    f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/test/physiology/*1hz.csv')

    os.makedirs(f'/work/abslab/emognition_2023_challenge/results/final_submission/scenario_{scenario}/fold_{fold}/test/annotations/',exist_ok=True)
    for test_x_file in test_x_files:
        file_name_split = test_x_file.split('/')[-1].split('_')
        sub, video = file_name_split[1],file_name_split[3]
        annote_path = f'annotations/sub_{sub}_vid_{video}.csv'
        annotation_file = f'/work/abslab/emognition_2023_challenge/results/final_submission/scenario_{scenario}/fold_{fold}/test/{annote_path}'
        if os.path.exists(annotation_file):
            annot_df = pd.read_csv(annotation_file)

        else:
            test_annotation_clean_path =test_x_file.split('phy')[0]+annote_path
            annot_df = pd.read_csv(test_annotation_clean_path)
        test_pred = model.predict(pd.read_csv(test_x_file)[x_columns].fillna(0).values)
        upsampled_preds = upsample_to_answer_shape(test_pred,len(annot_df))
        annot_df[y_columns[0]]=upsampled_preds
        annot_df.to_csv(annotation_file,index=None)

0


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


3


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# scenario 1

In [3]:
scenario = 1
y_columns=['arousal']
train_y_files = glob.glob(
    f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/train/annotations/*.csv')
train_x_files = glob.glob(
    f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/train/physiology/*1hz_zscored.csv')
model = ensemble.RandomForestRegressor()
x_columns = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets', 'trapezius_EMG_Raw',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Raw', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    'vid_num',
]

model = pipeline_helper.train_model_no_eval( 
    model,
    train_x_files,
    train_y_files,
    x_columns,
    y_columns,
    f'test_train_model_rf_scenario_{scenario}_fold_fizz_{y_columns[0]}'
)
test_x_files = glob.glob(
f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/test/physiology/*1hz.csv')

os.makedirs(f'/work/abslab/emognition_2023_challenge/results/final_submission/results/scenario_{scenario}/test/annotations/',exist_ok=True)
for test_x_file in test_x_files:
    file_name_split = test_x_file.split('/')[-1].split('_')
    sub, video = file_name_split[1],file_name_split[3]
    annote_path = f'annotations/sub_{sub}_vid_{video}.csv'
    annotation_file = f'/work/abslab/emognition_2023_challenge/results/final_submission/results/scenario_{scenario}/test/{annote_path}'
    if os.path.exists(annotation_file):
        annot_df = pd.read_csv(annotation_file)
    
    else:
        test_annotation_clean_path =test_x_file.split('phy')[0]+annote_path
        annot_df = pd.read_csv(test_annotation_clean_path)
    test_pred = model.predict(pd.read_csv(test_x_file)[x_columns].fillna(0).values)
    upsampled_preds = upsample_to_answer_shape(test_pred,len(annot_df))
    annot_df[y_columns[0]]=upsampled_preds
    annot_df.to_csv(annotation_file,index=None)

/work/abslab/emognition_2023_challenge/scripts/pipeline_helper.py:256: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train_x, train_y)


In [4]:
scenario = 1
y_columns=['valence']
train_y_files = glob.glob(
    f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/train/annotations/*.csv')
train_x_files = glob.glob(
    f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/train/physiology/*1hz_zscored.csv')
model = ensemble.RandomForestRegressor()
x_columns = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality','PPG_Peak_Height',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets', 'trapezius_EMG_Raw',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Raw', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    'vid_num',
]

model = pipeline_helper.train_model_no_eval( 
    model,
    train_x_files,
    train_y_files,
    x_columns,
    y_columns,
    f'test_train_model_rf_scenario_{scenario}_fold_fizz_{y_columns[0]}'
)
test_x_files = glob.glob(
f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/test/physiology/*1hz.csv')

os.makedirs(f'/work/abslab/emognition_2023_challenge/results/final_submission/results/scenario_{scenario}/test/annotations/',exist_ok=True)
for test_x_file in test_x_files:
    file_name_split = test_x_file.split('/')[-1].split('_')
    sub, video = file_name_split[1],file_name_split[3]
    annote_path = f'annotations/sub_{sub}_vid_{video}.csv'
    annotation_file = f'/work/abslab/emognition_2023_challenge/results/final_submission/results/scenario_{scenario}/test/{annote_path}'
    if os.path.exists(annotation_file):
        annot_df = pd.read_csv(annotation_file)
    
    else:
        test_annotation_clean_path =test_x_file.split('phy')[0]+annote_path
        annot_df = pd.read_csv(test_annotation_clean_path)
    test_pred = model.predict(pd.read_csv(test_x_file)[x_columns].fillna(0).values)
    upsampled_preds = upsample_to_answer_shape(test_pred,len(annot_df))
    annot_df[y_columns[0]]=upsampled_preds
    annot_df.to_csv(annotation_file,index=None)

/work/abslab/emognition_2023_challenge/scripts/pipeline_helper.py:256: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(train_x, train_y)


In [9]:
# scenario = 1
# test_x_files = glob.glob(
# f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/test/physiology/*1hz.csv')

# for test_x_file in test_x_files:
#     file_name_split = test_x_file.split('/')[-1].split('_')
#     sub, video = file_name_split[1],file_name_split[3]
#     annote_path = f'annotations/sub_{sub}_vid_{video}.csv'
#     annotation_file = f'/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_{scenario}/test/{annote_path}'
#     if os.path.exists(annotation_file):
#         annot_df = pd.read_csv(annotation_file)
    
#     else:
#         test_annotation_clean_path =test_x_file.split('phy')[0]+annote_path
#         annot_df = pd.read_csv(test_annotation_clean_path)
#     test_pred = model.predict(pd.read_csv(test_x_file)[x_columns].fillna(0).values)
#     upsampled_preds = upsample_to_answer_shape(test_pred,len(annot_df))
#     annot_df[y_columns[0]]=upsampled_preds
#     annot_df.to_csv(annotation_file,index=None)

# scenario 4

In [9]:
scenario = 4
y_columns=['valence']
for fold in range(2):
    print(fold)
    train_y_files = glob.glob(
        f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/train/annotations/*')
    train_x_files = glob.glob(
        f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/train/physiology/*1hz_zscored.csv')
    model = svm.SVR()
    x_columns = [ 
        'PPG_Rate', 
        'ECG_Rate','ECG_Quality','PPG_Peak_Height',
        'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
        'RSP_Amplitude', 'RSP_Rate',
        'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
        'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
        'sub_num',
        #'vid_num',
    ]

    model = pipeline_helper.train_model_no_eval( 
        model,
        train_x_files,
        train_y_files,
        x_columns,
        y_columns,
        f'test_train_model_svr_scenario_{scenario}_fold_{fold}_fizz_{y_columns[0]}'
    )
    test_x_files = glob.glob(
    f'/work/abslab/emognition_2023_challenge/data/scenario_{scenario}/fold_{fold}/test/physiology/*1hz.csv')

    os.makedirs(f'/work/abslab/emognition_2023_challenge/results/final_submission/results/scenario_{scenario}/fold_{fold}/test/annotations/',exist_ok=True)
    for test_x_file in test_x_files:
        file_name_split = test_x_file.split('/')[-1].split('_')
        sub, video = file_name_split[1],file_name_split[3]
        annote_path = f'annotations/sub_{sub}_vid_{video}.csv'
        annotation_file = f'/work/abslab/emognition_2023_challenge/results/final_submission/results/scenario_{scenario}/fold_{fold}/test/{annote_path}'
        if os.path.exists(annotation_file):
            annot_df = pd.read_csv(annotation_file)

        else:
            test_annotation_clean_path =test_x_file.split('phy')[0]+annote_path
            annot_df = pd.read_csv(test_annotation_clean_path)
        test_pred = model.predict(pd.read_csv(test_x_file)[x_columns].fillna(0).values)
        upsampled_preds = upsample_to_answer_shape(test_pred,len(annot_df))
        annot_df[y_columns[0]]=upsampled_preds
        annot_df.to_csv(annotation_file,index=None)

0


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1


/home/mcveigh.k/.local/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/test/annotations/sub_11_vid_1.csv',
 '/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/test/annotations/sub_11_vid_10.csv',
 '/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/test/annotations/sub_11_vid_11.csv',
 '/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/test/annotations/sub_11_vid_13.csv',
 '/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/test/annotations/sub_11_vid_14.csv',
 '/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/test/annotations/sub_11_vid_18.csv',
 '/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/test/annotations/sub_11_vid_20.csv',
 '/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/test/annotations/sub_11_vid_9.csv',
 '/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/test/annotati

In [26]:
for path in glob.glob('/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_1/*/*/*'):
    df = pd.read_csv(path)
    if sum(df.isna()['arousal']*1)>0:
        print(path)
    

In [28]:
for path in glob.glob('/work/abslab/emognition_2023_challenge/results/test_submissions/scenario_[2-4]/*/*/*/*'):
    df = pd.read_csv(path)
    if sum(df.isna()['arousal']*1)>0:
        print(path)